In [28]:
import tifffile
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import rc_context
from skimage import filters

from cellpose import models, core
from joblib import Parallel, delayed
import os

use_GPU = core.use_gpu()
print('>>>GPU activated? %d'%use_GPU)

2023-11-06 14:21:50,545 [INFO] ** TORCH CUDA version installed and working. **
>>>GPU activated? 1


In [29]:
from skimage.util import img_as_uint,img_as_float
from tqdm import tqdm
from skimage.measure import regionprops_table
import pandas as pd

In [30]:
# call logger_setup to have output of cellpose written
from cellpose.io import logger_setup
logger_setup()

creating new log file
2023-11-06 14:21:52,173 [INFO] WRITING LOG OUTPUT TO C:\Users\zfang38\.cellpose\run.log
2023-11-06 14:21:52,175 [INFO] 
cellpose version: 	2.2.2 
platform:       	win32 
python version: 	3.8.16 
torch version:  	2.0.1


# 3D nuclei segmentation

In [31]:
in_dir = r'..\..\coculture_diagonal\primed_cd8\00_registered\RNA'
fov_l = os.listdir(in_dir)
fov_l.sort()

In [32]:
im_l = []
for fov in tqdm(fov_l):
    im_l.append(tifffile.imread(os.path.join(in_dir,fov,'cycle1_mip_C1.tif')))

100%|██████████| 120/120 [00:23<00:00,  5.07it/s]


In [33]:
model = models.Cellpose(gpu=True,model_type='cyto2')

2023-11-06 14:26:10,619 [INFO] ** TORCH CUDA version installed and working. **
2023-11-06 14:26:10,621 [INFO] >>>> using GPU
2023-11-06 14:26:10,623 [INFO] >> cyto2 << model set to be used
2023-11-06 14:26:11,057 [INFO] >>>> model diam_mean =  30.000 (ROIs rescaled to this size during training)


In [34]:
channels = [0,0]
flow_threshold = 0.4
cellprob_threshold = 0.0
diameter = 100

In [35]:
masks, flows, styles, _ = model.eval(im_l, channels=channels, diameter=diameter,
                                     do_3D=False,flow_threshold=flow_threshold,
                                     cellprob_threshold=cellprob_threshold)

2023-11-06 14:26:15,942 [INFO] ~~~ FINDING MASKS ~~~
2023-11-06 14:29:59,570 [INFO] 40%|####      | 48/120 [03:43<04:25,  3.68s/it]
2023-11-06 14:29:59,998 [INFO] No cell pixels found.
2023-11-06 14:34:33,580 [INFO] 100%|##########| 120/120 [08:17<00:00,  4.15s/it]
2023-11-06 14:34:33,582 [INFO] >>>> TOTAL TIME 497.64 sec


In [36]:
out_dir = r'..\..\coculture_diagonal\primed_cd8\00_masks\all_cells'

In [37]:
for i,fov in enumerate(fov_l):
    save_name = os.path.join(out_dir, fov+'_mask.tif')
    tifffile.imwrite(save_name,masks[i],imagej=True)